# Question 1

Création de la table user_for_comparison et insertion des données


In [5]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

# Ajouter aux variables environnementales celles contenues dans le .env
load_dotenv(".env")

USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Connexion à PostgreSQL
conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432", dbname="app_auth")
conn.autocommit = True
cur = conn.cursor()

# Création de la table user_for_comparison
cur.execute("""
CREATE TABLE IF NOT EXISTS user_for_comparison (
    user_id SERIAL PRIMARY KEY,
    firstname VARCHAR(255),
    lastname VARCHAR(255),
    email VARCHAR(255) UNIQUE,
    username VARCHAR(255) UNIQUE,
    password VARCHAR(255),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

ALTER TABLE user_for_comparison
ADD CONSTRAINT username_length_check CHECK (LENGTH(username) > 8),
ADD CONSTRAINT password_length_check CHECK (LENGTH(password) > 8);
""")

# Initialiser Faker
fake = Faker()

# Ensemble pour suivre les emails et usernames uniques
used_emails = set()
used_usernames = set()

# Fonction pour générer une date aléatoire entre deux dates
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

# Dates de début et de fin
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 6, 30)

# Générer et insérer 100000 utilisateurs
for _ in range(100000):
    firstname = fake.first_name()
    lastname = fake.last_name()
    email = f"{firstname.lower()}.{lastname.lower()}@example.com"
    while email in used_emails:
        email = f"{firstname.lower()}.{lastname.lower()}{random.randint(1, 99)}@example.com"
    used_emails.add(email)

    username = f"{firstname.lower()}{lastname.lower()}{random.randint(100, 999)}"
    while username in used_usernames or len(username) <= 8:
        username = f"{firstname.lower()}{lastname.lower()}{random.randint(1000, 9999)}"
    used_usernames.add(username)

    password = fake.password(length=random.randint(9, 12))
    created_at = random_date(start_date, end_date).strftime("%Y-%m-%d")

    # Préparer et exécuter la requête SQL d'insertion
    try:
        query = """
        INSERT INTO user_for_comparison (firstname, lastname, email, username, password, created_at)
        VALUES (%s, %s, %s, %s, %s, %s);
        """
        cur.execute(query, (firstname, lastname, email, username, password, created_at))
    except psycopg2.errors.UniqueViolation:
        # Ignorer l'insertion en cas de violation de l'unicité
        continue

# Valider les insertions
conn.commit()

# Fermer la connexion
cur.close()
conn.close()


# Question 2

Mesurer le temps d'exécution

In [4]:
import psycopg2
import os
import random
import time
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(".env")

# Paramètres de connexion PostgreSQL
USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432", dbname="app_auth")
cur = conn.cursor()

# Récupérer tous les usernames de la table user_for_comparison
cur.execute("SELECT username FROM user_for_comparison")
usernames = [row[0] for row in cur.fetchall()]

# Choisir trois usernames aléatoirement
selected_usernames = random.sample(usernames, 3)

# Mesurer le temps d'exécution pour chaque username sélectionné et accumuler
total_time = 0
num_queries = len(selected_usernames)

for username in selected_usernames:
    start_time = time.time() # Commencer le chronomètre
    cur.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    result = cur.fetchone()
    end_time = time.time() # Arrêter le chronomètre
    execution_time = end_time - start_time # Calculer le temps d'exécution
    total_time += execution_time # Accumuler le temps total

# Calculer le temps moyen d'exécution
average_execution_time = total_time / num_queries

# Afficher le temps moyen d'exécution
print(f"Temps moyen d'exécution pour les usernames sélectionnés: {average_execution_time} seconds")

# Fermer la connexion
cur.close()
conn.close()


UndefinedFunction: ERREUR:  la fonction avg() n'existe pas
LINE 1: SELECT AVG(*) FROM user_for_comparison WHERE username = 'man...
               ^
HINT:  Aucune fonction ne correspond au nom donné et aux types d'arguments.
Vous devez ajouter des conversions explicites de type.


# Question 3

Index sur la colone username

In [12]:
import psycopg2
import os
from dotenv import load_dotenv

# Ajouter aux variables environnementales celles contenues dans le .env
load_dotenv(".env")

USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Connexion à PostgreSQL
conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432", dbname="app_auth")
cur = conn.cursor()

# Créer un index sur la colonne username
cur.execute("CREATE INDEX IF NOT EXISTS idx_username ON user_for_comparison (username);")

# Valider la création de l'index
conn.commit()

# Fermer la connexion
cur.close()
conn.close()


Test du temps d'execution

In [2]:
import psycopg2
import os
import random
import time
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(".env")

# Paramètres de connexion PostgreSQL
USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432", dbname="app_auth")
cur = conn.cursor()

# Récupérer tous les usernames de la table user_for_comparison
cur.execute("SELECT username FROM user_for_comparison")
usernames = [row[0] for row in cur.fetchall()]

# Choisir trois usernames aléatoirement
selected_usernames = random.sample(usernames, 3)

# Mesurer le temps d'exécution pour chaque username sélectionné et accumuler
total_time = 0
num_queries = len(selected_usernames)

for username in selected_usernames:
    start_time = time.time() # Commencer le chronomètre
    cur.execute("SELECT * FROM user_for_comparison WHERE username = %s", (username,))
    result = cur.fetchone()
    end_time = time.time() # Arrêter le chronomètre
    execution_time = end_time - start_time # Calculer le temps d'exécution
    total_time += execution_time # Accumuler le temps total

# Calculer le temps moyen d'exécution
average_execution_time = total_time / num_queries

# Afficher le temps moyen d'exécution
print(f"Temps moyen d'exécution pour les usernames sélectionnés: {average_execution_time} seconds")

# Fermer la connexion
cur.close()
conn.close()


Temps moyen d'exécution pour les usernames sélectionnés: 0.0013344287872314453 seconds
